<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 5
## Memory and State

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 5 — Memory State. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


We implement a tiny memory store with cosine similarity (NumPy) and a short summary function for the last N entries.

In [ ]:
%pip install -q numpy


#### What This Cell Does

This cell imports modules and sets up small helpers.


In [ ]:
import re  # import regular expressions
import numpy as np  # import NumPy for vectors

class Store:
    """Tiny bag-of-words memory with cosine similarity."""

    def __init__(self) -> None:
        self.entries = []  # store raw texts
        self.vocab = {}  # map token -> index

    def _tokens(self, text: str) -> list[str]:
        return re.findall(r'[A-Za-z0-9_]+', text.lower())  # tokenize text

    def _vector(self, text: str) -> np.ndarray:
        tokens = self._tokens(text)  # collect tokens
        for token in tokens:
            self.vocab.setdefault(token, len(self.vocab))  # add new token
        vec = np.zeros(len(self.vocab), dtype=float)  # allocate vector
        for token in tokens:
            vec[self.vocab[token]] += 1.0  # count occurrences
        norm = np.linalg.norm(vec) or 1.0  # avoid divide by zero
        return vec / norm  # normalize vector

    def _pad(
        self,
        entry_vec: np.ndarray,
        query_vec: np.ndarray,
    ) -> tuple[np.ndarray, np.ndarray]:
        size = max(len(entry_vec), len(query_vec))  # target length
        padded_entry = np.zeros(size, dtype=float)  # allocate entry buffer
        padded_query = np.zeros(size, dtype=float)  # allocate query buffer
        padded_entry[: len(entry_vec)] = entry_vec  # copy entry values
        padded_query[: len(query_vec)] = query_vec  # copy query values
        return padded_entry, padded_query  # equal-length vectors

    def add(self, text: str) -> None:
        self.entries.append(text)  # append new entry

    def top(self, query: str, k: int = 1):
        query_vec = self._vector(query)  # vectorize query
        scores = []  # collect scores
        for entry in self.entries:
            entry_vec = self._vector(entry)  # vectorize entry
            entry_vec, padded_query = self._pad(entry_vec, query_vec)  # pad lengths
            score = float(entry_vec @ padded_query)  # cosine similarity
            scores.append((entry, score))  # store score
        return sorted(scores, key=lambda pair: pair[1], reverse=True)[:k]  # top-k

    def summarize(self, n: int = 2) -> str:
        window = '\n'.join(self.entries[-n:])  # combine last notes
        highlights = [
            word
            for word in window.split()
            if word.isdigit() or (word[:1].isupper() and word[1:].islower())
        ]  # pick salient tokens
        return '\n'.join(highlights) or 'n/a'  # return highlight list


def demo() -> None:
    store = Store()  # create memory
    store.add('Calculate 2 and 3')  # add numeric note
    store.add('Email Alice the PDF')  # add named note
    print(store.top('add numbers', 1))  # show best match
    print(store.summarize(2))  # show highlights


demo()  # run demo


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>